# 24. Lidar point cloud cluster processing

This notebooks sets up files to process all Lidar point clouds on the Jassmin cluster

In [2]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2

In [3]:
import geopandas as gpd
import pandas as pd
import pathlib
import os
import datetime
from tqdm.autonotebook import tqdm

from src.constants import USER_PATH, PAISAGENSLIDAR_PATH, EBALIDAR_PATH, SCRATCH_PATH
from src.utils.os import list_content, latest_matching_file
from src.utils.slurm import sbatch_header, sbatch

from src.processing.slurm_jobcreator import lidR_gridmetrics_job, lidR_processing_job

/tmp/ipykernel_19600/760213292.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Helper functions

In [4]:
is_empty = lambda folder: not any(folder.iterdir())
contains = lambda folder, name: len(list(folder.glob(name))) > 0

## 1. Paisagenslidar - Cluster normalisation

In [5]:
paisagens_meta_path = latest_matching_file(PAISAGENSLIDAR_PATH / "*metadata")
paisagens_meta = gpd.read_file(paisagens_meta_path)

In [6]:
paisagens_paths = paisagens_meta.groupby("survey").first().path.apply(lambda x: pathlib.Path(x).parent)
paisagens_years = paisagens_meta.groupby("survey")[["year", "crs", "region", "plot_code", "creation_date"]].first()
paisagens_nfiles = paisagens_meta.groupby("survey").filename.count()
paisagens_size = paisagens_meta.groupby("survey")[["area (ha)",  "file_size (MB)", "points (Mio.)"]].sum()

In [7]:
paisagens_surveys = pd.concat([paisagens_paths, paisagens_years, paisagens_nfiles, paisagens_size], join="inner", axis=1)
paisagens_surveys["name"] = paisagens_surveys.apply(lambda x: f"{x.region}_{x.plot_code}_{x.year}", axis=1)
paisagens_surveys = paisagens_surveys.sort_values(by=["year", "region", "plot_code"])

### Optional
At this point, it may be prudent to test processing a single file to ensure that things are in good working order before issuing batch jobs.

The following code should complete without error.

Output files will be placed in the scratch directory (/work/scratch-nopw on JASMIN)



In [73]:
import subprocess
survey = paisagens_surveys[paisagens_surveys.name == "CUI_A01_2008"]

out_dir = SCRATCH_PATH / "Paisagenslidar" / "processed" / str(survey["year"].values[0]) / survey["name"].values[0]
print(survey['path'].values[0])
print(out_dir)

job_file = lidR_processing_job(lidar_path=(survey['path'].values[0]),
                                survey_name=survey['name'].values[0], 
                                out_dir = out_dir,
                                queue="short-serial",
                                max_time = datetime.time(23, 59, 0),
                                chunk_size=0,
                                buffer=50,
                                filter_quantile = 0.95,
                                filter_sensitivity = 1.1,
                                filter_gridsize= 10,
                                as_file=True, 
                                perform_check=False,
                                compress_intermediates=True, 
                                required_memory_per_node=15000,
                                )
    
subprocess.run('sh', job_file)


/gws/nopw/j04/forecol/data/Paisagenslidar/Mission_2008/CUI_A01_2008_LiDAR/CUI_A01_2008_laz
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/CUI_A01_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/CUI_A01_2008/slurm/slurm_CUI_A01_2008.sh
Submitted batch job 11755665
Submitted job for survey
CUI_A01_2008_laz    CUI_A01_2008
Name: name, dtype: object


### Normalisation: Create slurm job

You can check running jobs with the following command (Formatting string to show full job names and info):
```
$ squeue --format="%.18i %.9P %.30j %.8u %.8T %.10M %.9l %.6D %R"  | grep $USER
```

In [7]:
for idx, survey in tqdm(paisagens_surveys.iterrows(), total=len(paisagens_surveys)):

    out_dir = SCRATCH_PATH / "Paisagenslidar" / "processed" / str(survey["year"]) / survey["name"]
    # out_dir = PAISAGENSLIDAR_PATH / "processed" / str(survey["year"]) / survey["name"]

    # The processing job creates the following folders:
    # csf_ground: Classification of ground points
    # normalised: Height normalisation based on ground points
    # processed: Noise filtering; final results of normalisation
    # If the "processed" directory already exists and is non-empty, do not re-process the data
    if contains(out_dir, "processed") and not is_empty(out_dir / "processed"):
        continue
    
    job_file = lidR_processing_job(lidar_path=survey.path,
                                   survey_name=survey.name, 
                                   out_dir = out_dir,
                                   queue="short-serial",
                                   max_time = datetime.time(23, 59, 0),
                                   chunk_size=0,
                                   buffer=50,
                                   filter_quantile = 0.95,
                                   filter_sensitivity = 1.1,
                                   filter_gridsize= 10,
                                   as_file=True, 
                                   perform_check=False,
                                   compress_intermediates=True, 
                                   required_memory_per_node=15000,
                                  )
        
    print(job_file)
    break
    # Submit job
    sbatch(job_file)
    print(f"Submitted job for {survey['name']}")

  0%|          | 0/160 [00:00<?, ?it/s]

/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/DUC_A01_2008/slurm/slurm_DUC_A01_2008_laz.sh
Submitted batch job 11891408
Submitted job for DUC_A01_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/TAP_A01_2008/slurm/slurm_TAP_A01_2008_laz.sh
Submitted batch job 11891414
Submitted job for TAP_A01_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/TAP_A04_2008/slurm/slurm_TAP_A04_2008_laz.sh
Submitted batch job 11891420
Submitted job for TAP_A04_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2012/TAP_A02_2012/slurm/slurm_TAP_A02_2012_laz.sh
Submitted batch job 11891429
Submitted job for TAP_A02_2012
/work/scratch-nopw/forecol/Paisagenslidar/processed/2014/BAR_A01_2014/slurm/slurm_BAR_A01_2014_laz.sh
Submitted batch job 11891439
Submitted job for BAR_A01_2014
/work/scratch-nopw/forecol/Paisagenslidar/processed/2014/CAU_A01_2014/slurm/slurm_CAU_A01_2014_laz.sh
Submitted batch job 11891441
Submitted job for CAU_A01_2014
/work/scratch-nopw/forecol/P

### Metrics: Create slurm job

In [12]:
unavailable = []
for idx, survey in paisagens_surveys.iterrows():
    
    if survey["path"] is None or pd.isna(survey["path"]):
        continue
    
    if os.getenv("USE_SCRATCH"):
        out_dir = SCRATCH_PATH / "Paisagenslidar" / "processed" / str(survey["year"]) / survey["name"]
    else:
        out_dir = PAISAGENSLIDAR_PATH / "processed" / str(survey["year"]) / survey["name"]
    
    # The processing job stores its output in the folder grid_metrics.
    # If this folder already exists and is not empty, do not reprocess.
    # TODO: Move this check intor slurm_jobcreator.py
    if contains(out_dir, "grid_metrics") and not is_empty(out_dir / "grid_metrics"):
        continue
    
    # If the LiDAR data was not successfully normalised in the step above,
    # Print a warning and continue
    if not (out_dir / "processed").exists() or is_empty(out_dir / "processed"):
        unavailable.append(survey["name"])
        continue
    
    job_file = lidR_gridmetrics_job(lidar_path=out_dir/"processed",
                                    survey_name=survey.name, 
                                    out_dir=out_dir,
                                    metrics = {"n_points": [1],
                                               "n_pulses": [1],
                                               "n_ground_points": [1],
                                               "max": [10, 50],
                                               "quantile_0.05": [10, 50],
                                               "quantile_0.1": [10, 50],
                                               "kurtosis": [10, 50],
                                               "mean": [10, 50],
                                               "interquartile_range": [10, 50],
                                               "longo_biomass": [10, 50], 
                                               "canopy_height": [1]

                                    },
                                    buffer = 10,
                                    chunk_size = 0,
                                    as_file = True,
                                    overwrite=False,
                                    queue="short-serial",
                                    required_memory_per_node=8000,    
                                    max_time = datetime.time(10, 0, 0)
                                   )
    print(job_file)
    break
    # Submit job
    sbatch(job_file)
    print(f"Submitted job for {survey['name']}")

if len(unavailable) > 0:
    print("Could not find normalized input data for the following {} surveys:".format(len(unavailable)))
    for s in unavailable:
        print(s)

/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/CUI_A01_2008/slurm/slurm_CUI_A01_2008_laz_grid_metrics.sh
Submitted batch job 11983452
Submitted job for CUI_A01_2008
DUC_A01_2008 not available.
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/PRE_A01_2008/slurm/slurm_PRE_A01_2008_laz_grid_metrics.sh
Submitted batch job 11983453
Submitted job for PRE_A01_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/PRE_A02_2008/slurm/slurm_PRE_A02_2008_laz_grid_metrics.sh
Submitted batch job 11983454
Submitted job for PRE_A02_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/PRE_A04_2008/slurm/slurm_PRE_A04_2008_laz_grid_metrics.sh
Submitted batch job 11983455
Submitted job for PRE_A04_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/PRE_A05_2008/slurm/slurm_PRE_A05_2008_laz_grid_metrics.sh
Submitted batch job 11983456
Submitted job for PRE_A05_2008
/work/scratch-nopw/forecol/Paisagenslidar/processed/2008/PRE_A06_2008/slurm/slurm_PRE_A06_2008_la

-----------

## 2. EBA - Cluster normalisation

In [13]:
eba_meta_path = latest_matching_file(EBALIDAR_PATH / "*metadata")
eba_meta = gpd.read_file(eba_meta_path)

In [14]:
eba_laz = {path.name: {"path": path,
                       "name": path.name.split(".")[0],
                       "size": round(path.stat().st_size / 1024**2,2)
                      } 
           for path in (EBALIDAR_PATH / "laz_EBA").glob("*.laz")
          }

In [15]:
df = pd.DataFrame(eba_laz).T.convert_dtypes()
df = df.rename(columns={"size": "size (MB)"})
df["reduced_filename"] = df["name"] + ".laz"

In [16]:
eba_new_meta = pd.merge(eba_meta.drop(columns=["path", "size (MB)", "name"]), df, how="outer", on="reduced_filename")
eba_new_meta["size (MB)"] = eba_new_meta["size (MB)"].astype(float)
eba_new_meta.index = eba_new_meta["reduced_filename"]
eba_new_meta.index.name = "survey"
eba_new_meta = eba_new_meta.sort_index()
eba_new_meta

,survey,transect,field_data,hyperspect,obs,campaign,criteria,datafile,elev_maxim,elev_mean,...,canopy_rel,elev_sqrt_,elev_curt_,profile_ar,random,reduced_filename,geometry,path,name,size (MB)
survey,,,,,,,,,,,,,,,,,,,,,
NP_T-0001.laz,NP_T-0001.laz,0001,None,None,None,1.0,PRODES+TERRACLASS,NP_T-0001_dn_g_n_ch1_5.laz,38.12,13.4278,...,0.402270,14.9651,15.9663,50.9332,yes,NP_T-0001.laz,"POLYGON ((4113515.087 10124216.741, 4113517.77...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-0001,233.47
NP_T-0002.laz,NP_T-0002.laz,0002,None,None,None,1.0,PRODES+TERRACLASS,NP_T-0002_dn_g_n_ch1_5.laz,53.67,15.6308,...,0.357771,18.2027,19.9904,41.2544,yes,NP_T-0002.laz,"POLYGON ((4051446.715 10275374.329, 4051471.18...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-0002,254.80
NP_T-0003.laz,NP_T-0003.laz,0003,None,None,None,1.0,PRODES+TERRACLASS,NP_T-0003_dn_g_n_ch1_5.laz,72.00,16.3030,...,0.327655,18.6746,20.6112,37.3803,yes,NP_T-0003.laz,"POLYGON ((4030556.091 10289424.141, 4030585.79...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-0003,204.94
NP_T-0004.laz,NP_T-0004.laz,0004,None,None,None,1.0,PRODES+TERRACLASS,NP_T-0004_dn_g_n_ch1_5.laz,58.54,13.6670,...,0.297323,16.5975,18.8821,33.5121,yes,NP_T-0004.laz,"POLYGON ((4117057.589 10328223.327, 4117057.60...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-0004,208.09
NP_T-0005.laz,NP_T-0005.laz,0005,None,None,None,1.0,PRODES+TERRACLASS,NP_T-0005_dn_g_n_ch1_5.laz,62.72,15.2655,...,0.602625,17.4348,19.1750,38.3784,yes,NP_T-0005.laz,"POLYGON ((4073980.272 10342069.880, 4073985.18...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-0005,234.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_T-2001.laz,NP_T-2001.laz,2001,None,None,None,2.0,Directed,NP_T-2001_dn_g_n_ch1_5.laz,75.70,19.9900,...,0.544592,22.7049,24.3514,48.3164,no,NP_T-2001.laz,"POLYGON ((4687939.276 9807283.147, 4687940.579...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-2001,225.71
NP_T-2002.laz,NP_T-2002.laz,2002,None,None,None,2.0,Directed,NP_T-2002_dn_g_n_ch1_5.laz,65.02,21.2876,...,0.451701,23.6641,25.3670,47.6701,no,NP_T-2002.laz,"POLYGON ((4015043.537 8959928.665, 4015049.131...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-2002,190.41
NP_T-2004.laz,NP_T-2004.laz,2004,None,None,None,2.0,Directed,NP_T-2004_dn_g_n_ch1_5.laz,38.56,13.5921,...,0.445896,15.4306,16.6037,48.4385,no,NP_T-2004.laz,"POLYGON ((4965389.690 8676632.304, 4965375.149...",/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-2004,185.83


In [17]:
#eba_new_meta.to_file(eba_meta_path.parent / f"eba_file_metadata_v{datetime.date.today()}", driver="GPKG")

In [17]:
smallest = df.iloc[df["size (MB)"].values.argmin():df["size (MB)"].values.argmin()+1]
smallest

,path,name,size (MB),reduced_filename
NP_T-1041_03.laz,/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA/N...,NP_T-1041_03,53.26,NP_T-1041_03.laz


### Normalisation: Create slurm job

In [20]:
for idx, survey in tqdm(eba_new_meta.iterrows(), total=len(eba_new_meta)):
    
    if survey["path"] is None or pd.isna(survey["path"]):
        continue
    
    if os.getenv("USE_SCRATCH"):
        out_dir = SCRATCH_PATH / "EBA_lidar" / "laz_EBA_processed" / survey["name"]
    else:
        out_dir = EBALIDAR_PATH / "laz_EBA_processed" / survey["name"]
    
    if contains(out_dir, "processed") and not is_empty(out_dir / "processed"):
        continue
    #else:
        #print("Not processed:", survey["name"])
        #continue
    
    job_file = lidR_processing_job(lidar_path=survey.path,
                                   survey_name = survey.name,
                                   out_dir = out_dir,
                                   queue="short-serial",
                                   max_time = datetime.time(23, 59, 0),
                                   chunk_size=1000,
                                   buffer=50,
                                   filter_quantile = 0.95,
                                   filter_sensitivity = 1.1,
                                   filter_gridsize= 10,
                                   as_file=True, 
                                   perform_check=False,
                                   compress_intermediates=True, 
                                   required_memory_per_node=16000,
                                  )
    
    print(job_file)
    break
    # Submit job
    sbatch(job_file)
    print(f"Submitted job for {survey['name']}")

  0%|          | 0/930 [00:00<?, ?it/s]

/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0001/slurm/slurm_NP_T-0001.laz.sh
Submitted batch job 11984310
Submitted job for NP_T-0001
/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0002/slurm/slurm_NP_T-0002.laz.sh
Submitted batch job 11984311
Submitted job for NP_T-0002
/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0003/slurm/slurm_NP_T-0003.laz.sh
Submitted batch job 11984312
Submitted job for NP_T-0003
/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0004/slurm/slurm_NP_T-0004.laz.sh
Submitted batch job 11984313
Submitted job for NP_T-0004
/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0005/slurm/slurm_NP_T-0005.laz.sh
Submitted batch job 11984314
Submitted job for NP_T-0005
/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0006/slurm/slurm_NP_T-0006.laz.sh
Submitted batch job 11984315
Submitted job for NP_T-0006
/work/scratch-nopw/forecol/EBA_lidar/laz_EBA_processed/NP_T-0007/slurm/slurm_NP_T-0007.laz.sh


### Metrics: Create slurm job

In [21]:
unavailable = []
for idx, survey in tqdm(eba_new_meta.iterrows(), total=len(eba_new_meta)):
    
    if survey["path"] is None or pd.isna(survey["path"]):
        continue
    
    if os.getenv("USE_SCRATCH"):
        out_dir = SCRATCH_PATH / "EBA_lidar" / "laz_EBA_processed" / survey["name"]
    else: 
        out_dir = EBALIDAR_PATH / "laz_EBA_processed" / survey["name"]
    
    # The processing job stores its output in the folder grid_metrics.
    # If this folder already exists and is not empty, do not reprocess.
    # TODO: Move this check into slurm_jobcreator.py
    if contains(out_dir, "grid_metrics") and not is_empty(out_dir / "grid_metrics"):
        unavailable.append(survey.name)
        continue
    
    job_file = lidR_gridmetrics_job(# File params
                                    lidar_path=(out_dir / "processed"),
                                    survey_name=survey.name, 
                                    out_dir=out_dir,
                                    # Processing params
                                    metrics = {"point_density": [1],
                                               "pulse_density": [1],
                                               "ground_point_density": [1],
                                               "n_points": [1],
                                               "n_pulses": [1],
                                               "n_ground_points": [1],
                                               "max": [10, 50],
                                               "quantile_0.05": [10, 50],
                                               "quantile_0.1": [10, 50],
                                               "kurtosis": [10, 50],
                                               "mean": [10, 50],
                                               "interquartile_range": [10, 50],
                                               "longo_biomass": [10, 50], 
                                               "canopy_height": [1, 2]

                                    },
                                    buffer = 10,
                                    chunk_size = 0,
                                    as_file = False,

                                    # Queue params
                                    queue="short-serial",
                                    max_time = datetime.time(10, 0, 0),
                                    #expected_time = datetime.time(1, 30, 0),
                                    required_memory_per_node=8000,
                               )
    print(job_file)
    # Submit job
    break
    sbatch(job_file)
    print(f"Submitted job for {survey['name']}")

if len(unavailable) > 0:
    print("Could not find normalized input data for the following {} surveys:".format(len(unavailable)))
    for s in unavailable:
        print(s)

  0%|          | 0/930 [00:00<?, ?it/s]

#!/bin/bash
#SBATCH --partition=short-serial
#SBATCH --time=10:00:00  # Set a timeout for the job in HH:MM:SS
#SBATCH --mem=8000 # Set the amount of memory for per node in MB.
#SBATCH --job-name=NP_T-0001_grid_metrics
#SBATCH --chdir=/gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA_processed/NP_T-0001/slurm
#SBATCH --output=slurm_NP_T-0001_metrics-%j.out
#SBATCH --error=slurm_NP_T-0001_metrics-%j.err


# --- TEMPLATE AUTOGENERATED ---


echo "==================== Activating conda environment ===================="
source ~/.bashrc
conda activate r4_env

echo "==================== Checking R version ===================="
echo "$(which R)"
echo "$(R --version)"

echo "==================== Analysing file ===================="
echo "template generation time: 2021-06-24 13:02:05.482052"
echo "survey: NP_T-0001"
echo "path: /gws/nopw/j04/forecol/data/EBA_lidar/laz_EBA_processed/NP_T-0001/processed"
echo "metrics: {'point_density': [1], 'pulse_density': [1], 'ground_point_density': [1], 'n_points'

----------------

# To be deleted: Expedited processing for relevant files

In [119]:
relevant = [
    "NP_T-0408.laz",
    "NP_T-0859.laz",
    "NP_T-0145.laz",
    "NP_T-0751.laz",
    "NP_T-2002.laz",
    "NP_T-0416.laz",
    "NP_T-0669.laz",
]

for idx, survey in eba_new_meta.loc[relevant].iterrows():
    
    #if survey["path"] is None or pd.isna(survey["path"]):
    #    continue
    
    out_dir = EBALIDAR_PATH / "laz_EBA_processed" / survey["name"]
    
    job_file = create_lidar_gridmetrics_job(idx, 
                                survey, 
                                out_dir,
                                metrics = {"point_density": [1],
                                           "pulse_density": [1],
                                           "ground_point_density": [1],
                                           "n_points": [1],
                                           "n_pulses": [1],
                                           "n_ground_points": [1],
                                           "max": [10, 50],
                                           "quantile_0.05": [10, 50],
                                           "quantile_0.1": [10, 50],
                                           "kurtosis": [10, 50],
                                           "mean": [10, 50],
                                           "interquartile_range": [10, 50],
                                           "longo_biomass": [10, 50], 
                                           "canopy_height": [1, 2]
                                
                                },
                                buffer = 10,
                                chunk_size = 0,
                                as_file = True,
                                overwrite=False,
                                queue="test",
                                
                               )
    print(job_file)
    break
    # Submit job
    #sbatch(job_file)
    print(f"Submitted job for {survey['name']}")

Submitted job for NP_T-0408
Submitted job for NP_T-0859
Submitted job for NP_T-0145
Submitted job for NP_T-0751
Submitted job for NP_T-2002
Submitted job for NP_T-0416
Submitted job for NP_T-0669


In [11]:
paisagens_relevant = [
    "JAM_A03_2013_laz",
    "JAM_A02_2013_laz",
    "JAM_A02a_2014_laz",
    "JAM_A02c_2015_LiDAR",
    "SAN_A02_2014_laz",
    
]

for idx, survey in paisagens_surveys.loc[paisagens_relevant][4::].iterrows():
    
    #if survey["path"] is None or pd.isna(survey["path"]):
    #    continue
    
    out_dir = PAISAGENSLIDAR_PATH / "processed" / str(survey["year"]) / survey["name"]
    
    if contains(out_dir, "processed") and not is_empty(out_dir / "processed"):
        pass
    else:
        print(survey["name"], "not available.")
        continue
    
    job_file = lidR_gridmetrics_job(survey, 
                                    out_dir,
                                    metrics = {"n_points": [1],
                                               "n_pulses": [1],
                                               "n_ground_points": [1],
                                               "max": [10, 50],
                                               "quantile_0.05": [10, 50],
                                               "quantile_0.1": [10, 50],
                                               "kurtosis": [10, 50],
                                               "mean": [10, 50],
                                               "interquartile_range": [10, 50],
                                               "longo_biomass": [10, 50], 
                                               "canopy_height": [1]

                                    },
                                    buffer = 10,
                                    chunk_size = 0,
                                    as_file = False,
                                    overwrite=False,
                                    queue="test",
                                    required_memory_per_node=8000,    
                                    max_time = datetime.time(10, 0, 0)
                                   )
    print(job_file)
    break
    # Submit job
    sbatch(job_file)
    print(f"Submitted job for {survey['name']}")

#!/bin/bash
#SBATCH --partition=test
#SBATCH --time=10:00:00  # Set a timeout for the job in HH:MM:SS
#SBATCH --mem=8000 # Set the amount of memory for per node in MB.
#SBATCH --job-name=SAN_A02_2014_grid_metrics
#SBATCH --chdir=/gws/nopw/j04/forecol/data/Paisagenslidar/processed/2014/SAN_A02_2014/slurm
#SBATCH --output=slurm_SAN_A02_2014_metrics-%j.out
#SBATCH --error=slurm_SAN_A02_2014_metrics-%j.err


# --- TEMPLATE AUTOGENERATED ---


echo "==================== Activating conda environment ===================="
source ~/.bashrc
conda activate r4_env

echo "==================== Checking R version ===================="
echo "$(which R)"
echo "$(R --version)"

echo "==================== Analysing file ===================="
echo "template generation time: 2021-06-24 18:16:43.778255"
echo "survey: SAN_A02_2014"
echo "path: /gws/nopw/j04/forecol/data/Paisagenslidar/processed/2014/SAN_A02_2014/processed"
echo "metrics: {'n_points': [1], 'n_pulses': [1], 'n_ground_points': [1], 'max': [10,

----

# Legacy

In [5]:
def create_lidar_processing_job(idx, 
                                survey, 
                                r_file,
                                out_dir,
                                buffer = 50,
                                chunk_size = 0,
                                filter_quantile = 0.95, 
                                filter_sensitivity = 1.1,
                                filter_gridsize = 10,
                                compress_intermediates = True,
                                save_intermediates = True,
                                perform_check = True,
                                as_file = True,
                                **kwargs
                               ):
    # Path for lidar data
    lidar_path = pathlib.Path(survey["path"])
    assert lidar_path.exists()
    
    # Set up save prefix
    save_prefix = survey["name"] + "_"

    # Create working directory path
    slurm_dir = out_dir / "slurm"
    slurm_dir.mkdir(parents=True, exist_ok=True)

    # Create sbatch header
    header = sbatch_header(
                  job_name = idx + "-->" + survey["name"],
                  working_directory=slurm_dir,
                  output_file_pattern=f"slurm_{survey['name']}-%j.out",
                  error_file_pattern=f"slurm_{survey['name']}-%j.err",
                  **kwargs)

    body = seperator("Activating conda environment")
    body += "source ~/.bashrc\n"
    body += "conda activate r4_env\n"
    body += seperator("Checking R version")
    body += echo("$(which R)")
    #body += "echo $(which R)\n"
    body += echo("$(R --version)")
    body += seperator("Analysing file")
    body += echo(f"template generation time: {datetime.datetime.now()}")
    body += echo(f"survey: {survey['name']}")
    body += echo(f"path: {lidar_path}")
    body += seperator("Executing R script")
    body += (f"Rscript --verbose --no-save {r_file}" 
             f" --lidar_path={lidar_path}" 
             f" --save_path={out_dir}"
             f" --buffer={buffer}"
             f" --chunk_size={chunk_size}"
             f" --filter_quantile={filter_quantile}"
             f" --filter_sensitivity={filter_sensitivity}"
             f" --filter_gridsize={filter_gridsize}"
             f" --save_prefix={save_prefix}"
             f" --perform_check={'TRUE' if perform_check else 'FALSE'}"
             f" --save_intermediates={'TRUE' if save_intermediates else 'FALSE'}"
             f" --compress_intermediates={'TRUE' if compress_intermediates else 'FALSE'}"
            )

    if as_file:
        job_file = slurm_dir / f"slurm_{survey['name']}.sh"
        with open(job_file, "w") as file:
            file.write(header + body)
        return job_file
    else: 
        return header+body

In [13]:
ALLOWED_METRICS = ("point_density", "pulse_density", "ground_point_density",
                   "n_points", "n_pulses", "n_ground_points",
                   "max", "standard_dev", "mask", "mean", "kurtosis", 
                   "interquartile_range", "quantile", "longo_biomass")

r_gridmetrics_file = "/home/users/svm/Code/gedi_biomass_mapping/src/processing/lidar_gridmetrics_script.R"
r_canopy_height_file = "/home/users/svm/Code/gedi_biomass_mapping/src/processing/lidar_canopy_height_script.R"

def create_lidar_gridmetrics_job(idx, 
                                survey, 
                                out_dir,
                                metrics,
                                buffer = 10,
                                chunk_size = 0,
                                save_prefix="",
                                check_index=True,
                                overwrite=False,
                                as_file = False,
                                **kwargs
                               ):
    # Path for lidar data
    lidar_path = pathlib.Path(out_dir / "processed")
    assert lidar_path.exists()
    
    # Set up save prefix
    save_prefix = survey["name"] + "_"

    # Create working directory path
    slurm_dir = out_dir / "slurm"
    slurm_dir.mkdir(parents=True, exist_ok=True)
    grid_metrics_dir = out_dir / "grid_metrics"
    grid_metrics_dir.mkdir(parents=True, exist_ok=True)

    # Create sbatch header
    header = sbatch_header(
                  job_name = survey["name"] + "_grid_metrics",
                  working_directory=slurm_dir,
                  output_file_pattern=f"slurm_{survey['name']}_metrics-%j.out",
                  error_file_pattern=f"slurm_{survey['name']}_metrics-%j.err",
                  **kwargs)

    body = seperator("Activating conda environment")
    body += "source ~/.bashrc\n"
    body += "conda activate r4_env\n"
    body += seperator("Checking R version")
    body += echo("$(which R)")
    body += echo("$(R --version)")
    body += seperator("Analysing file")
    body += echo(f"template generation time: {datetime.datetime.now()}")
    body += echo(f"survey: {survey['name']}")
    body += echo(f"path: {lidar_path}")
    body += echo(f"metrics: {metrics}")
    body += seperator("Executing R script")
    
    if "canopy_height" in metrics.keys():
        for gridsize in metrics.pop("canopy_height"):
            body += f"\n# `Canopy height` at grid size `{gridsize}m`\n"
            body += (f"Rscript --verbose --no-save {r_canopy_height_file}" 
                     f" --lidar_path={lidar_path}"
                     f" --gridsize={gridsize}" 
                     f" --save_path={out_dir}"
                     f" --buffer={buffer}"
                     f" --chunk_size={chunk_size}"
                     f" --save_prefix={save_prefix}"
                     f" --overwrite={'TRUE' if overwrite else 'FALSE'}"
                     f" --check_index={'TRUE' if check_index else 'FALSE'}"
                     "\n"
                    )
    
    for metric in metrics.keys():
        assert metric in ALLOWED_METRICS or "quantile_" in metric, f"{metric} not allowed."
        for gridsize in metrics[metric]:
            body += f"\n# Metric `{metric}` at grid size `{gridsize}m`\n"
            body += (f"Rscript --verbose --no-save {r_gridmetrics_file}" 
                     f" --lidar_path={lidar_path}" 
                     f" --metric={metric if 'quantile' not in metric else 'quantile'}"
                     f" --gridsize={gridsize}" 
                     f" --save_path={out_dir}"
                     f" --buffer={buffer}"
                     f" --chunk_size={chunk_size}"
                     f" --save_prefix={save_prefix}"
                     f" --overwrite={'TRUE' if overwrite else 'FALSE'}"
                     f" --check_index={'TRUE' if check_index else 'FALSE'}"
                     f"{' --quantile='+str(metric.split('_')[-1]) if 'quantile' in metric else ''}"
                     "\n"
                    )

    if as_file:
        job_file = slurm_dir / f"slurm_{survey['name']}_grid_metrics.sh"
        with open(job_file, "w") as file:
            file.write(header + body)
        return job_file
    else: 
        return header+body